In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('http://www.kakapart.com/oem/partscrossinterchange/centric-' + input_.loc[a, 'Sku'].replace('Part # ', '').replace('.', '') + '.html',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@class="os-main-table"]/tbody/tr[@class="os-row"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_oe = {}
                for tr in list_tr:
                    factory = tr.xpath('./td[1]/text()')[0].strip()
                    cross = tr.xpath('./td[3]/text()')[0].strip()
                    if cross == 'Direct Cross Interchange':
                        key = factory + '_Direct'
                    elif cross == 'Indirect Cross Interchange':
                        key = factory + '_Indirect'
                    else:
                        key = factory + '_NULL'

                    if key not in dict_oe:
                        dict_oe[key] = [tr.xpath('./td[2]/text()')[0].strip()]
                    else:
                        dict_oe[key].append(tr.xpath('./td[2]/text()')[0].strip())
                        dict_oe[key] = sorted(list(set(dict_oe[key])))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Sku': input_.loc[a, 'Sku']}])

                for key in dict_oe:
                    df_temp.loc[0, 'OE_' + key] = ';'.join(dict_oe[key])
    
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Sku': input_.loc[a, 'Sku']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Sku'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Sku'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./oe-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Sku'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./oe_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：283

[ok] - Part # 100.05940
[尝试次数：1] - [剩余数量：263] - [当前时间：11:12:45]

[ok] - Part # 100.08770
[尝试次数：1] - [剩余数量：262] - [当前时间：11:12:45]

[ok] - Part # 100.07200
[尝试次数：1] - [剩余数量：261] - [当前时间：11:12:45]

[ok] - Part # 100.06380
[尝试次数：1] - [剩余数量：260] - [当前时间：11:12:45]

[ok] - Part # 100.09154
[尝试次数：1] - [剩余数量：259] - [当前时间：11:12:45]

[ok] - Part # 100.01451
[尝试次数：1] - [剩余数量：258] - [当前时间：11:12:45]

[ok] - Part # 100.02900
[尝试次数：1] - [剩余数量：257] - [当前时间：11:12:45]

[ok] - Part # 100.07400
[尝试次数：1] - [剩余数量：256] - [当前时间：11:12:45]

[ok] - Part # 100.09722
[尝试次数：1] - [剩余数量：255] - [当前时间：11:12:45]

[ok] - Part # 100.09780
[尝试次数：1] - [剩余数量：254] - [当前时间：11:12:45]

[ok] - Part # 100.00310
[尝试次数：1] - [剩余数量：253] - [当前时间：11:12:45]

[ok] - Part # 100.04030
[尝试次数：1] - [剩余数量：252] - [当前时间：11:12:45]

[ok] - Part # 100.08760
[尝试次数：1] - [剩余数量：251] - [当前时间：11:12:46]

[ok] - Part # 100.06922
[尝试次数：1] - [剩余数量：250] - [当前时间：11:12:46]

[ok] - Part # 100.06400
[尝试次数：1] - [剩余数量：249] - [当前时间：11:12:46]

[ok] - Part # 10

[ok] - Part # 102.02210
[尝试次数：12] - [剩余数量：137] - [当前时间：11:13:17]

[ok] - Part # 102.02280
[尝试次数：6] - [剩余数量：136] - [当前时间：11:13:18]

[ok] - Part # 102.02060
[尝试次数：13] - [剩余数量：135] - [当前时间：11:13:18]

[ok] - Part # 102.01740
[尝试次数：26] - [剩余数量：134] - [当前时间：11:13:19]

[ok] - Part # 102.02250
[尝试次数：13] - [剩余数量：133] - [当前时间：11:13:19]

[ok] - Part # 102.02560
[尝试次数：6] - [剩余数量：132] - [当前时间：11:13:20]

[ok] - Part # 102.02570
[尝试次数：3] - [剩余数量：131] - [当前时间：11:13:20]

[ok] - Part # 102.02620
[尝试次数：4] - [剩余数量：130] - [当前时间：11:13:22]

[ok] - Part # 102.01910
[尝试次数：31] - [剩余数量：129] - [当前时间：11:13:22]

[ok] - Part # 102.01451
[尝试次数：53] - [剩余数量：128] - [当前时间：11:13:22]

[ok] - Part # 102.02610
[尝试次数：4] - [剩余数量：127] - [当前时间：11:13:22]

[ok] - Part # 102.02740
[尝试次数：1] - [剩余数量：126] - [当前时间：11:13:22]

[ok] - Part # 102.02750
[尝试次数：1] - [剩余数量：125] - [当前时间：11:13:23]

[ok] - Part # 102.02400
[尝试次数：13] - [剩余数量：124] - [当前时间：11:13:23]

[ok] - Part # 102.02770
[尝试次数：2] - [剩余数量：123] - [当前时间：11:13:23]

[ok] - Part # 102.

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_82967/3677071437.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp.loc[0, 'OE_' + key] = ';'.join(dict_oe[key])
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_82967/3677071437.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_temp.loc[0, 'OE_' + key] = ';'.join(dict_oe[key])
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_82967/3677071437.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of call

[ok] - Part # 102.03400
[尝试次数：22] - [剩余数量：59] - [当前时间：11:13:46]

[ok] - Part # 102.03870
[尝试次数：1] - [剩余数量：58] - [当前时间：11:13:47]

[ok] - Part # 102.03680
[尝试次数：7] - [剩余数量：57] - [当前时间：11:13:47]

[ok] - Part # 102.03750
[尝试次数：6] - [剩余数量：56] - [当前时间：11:13:47]

[ok] - Part # 102.03340
[尝试次数：28] - [剩余数量：55] - [当前时间：11:13:48]

[ok] - Part # 102.03810
[尝试次数：6] - [剩余数量：54] - [当前时间：11:13:48]

[ok] - Part # 102.03890
[尝试次数：3] - [剩余数量：53] - [当前时间：11:13:49]

[ok] - Part # 102.03700
[尝试次数：14] - [剩余数量：52] - [当前时间：11:13:49]

[ok] - Part # 102.03830
[尝试次数：5] - [剩余数量：51] - [当前时间：11:13:49]

[ok] - Part # 102.03780
[尝试次数：10] - [剩余数量：50] - [当前时间：11:13:50]

[ok] - Part # 102.03501
[尝试次数：22] - [剩余数量：49] - [当前时间：11:13:50]

[ok] - Part # 102.03770
[尝试次数：10] - [剩余数量：48] - [当前时间：11:13:50]

[ok] - Part # 102.03910
[尝试次数：7] - [剩余数量：47] - [当前时间：11:13:50]

[ok] - Part # 102.04010
[尝试次数：3] - [剩余数量：46] - [当前时间：11:13:51]

[ok] - Part # 102.03730
[尝试次数：16] - [剩余数量：45] - [当前时间：11:13:51]

[ok] - Part # 102.03940
[尝试次数：12]